## Importations des données et modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
import pandas as pd
import nltk
from nltk import word_tokenize
nltk.download('punkt_tab')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import spacy
import string
import re
from nltk.probability import FreqDist
from nltk.util import ngrams
# Téléchargement du modèle spaCy en ligne de commande (à exécuter une seule fois)
!python -m spacy download fr_core_news_md
nlp = spacy.load('fr_core_news_md')
nltk.download('stopwords')

Mounted at /content/drive


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 16.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# On importe les tweetsen csv

bdd_L_link = "/content/drive/MyDrive/projet/Tweet2.csv"
bdd_D_link = "/content/drive/MyDrive/projet/Tweet1.csv"
Tweet_csv_L = pd.read_csv(bdd_L_link, on_bad_lines='skip', sep=';')
Tweet_csv_D = pd.read_csv(bdd_D_link, on_bad_lines='skip',sep=';')

In [ ]:
# On réindexe et supprimme les lignes

Tweet_csv_L=Tweet_csv_L[['data__id','data__text']]
Tweet_csv_D=Tweet_csv_D[['data__id','data__text']]

Tweet_csv_L=(Tweet_csv_L.dropna(subset='data__text')).reset_index(drop=False)
Tweet_csv_D=(Tweet_csv_D.dropna(subset='data__text')).reset_index(drop=False)
Tweet_csv_L=(Tweet_csv_L[Tweet_csv_L['data__text']!='data__text'])[['index','data__text']]
Tweet_csv_D=(Tweet_csv_D[Tweet_csv_D['data__text']!='data__text'])[['index','data__text']]
#print(Tweet_csv_D.head(10))

In [ ]:
# On créé un data set d'entraînemnt et un autre de test

Tweet_csv_L_train, Tweet_csv_L_test = train_test_split(Tweet_csv_L, test_size=0.2, random_state=42)
Tweet_csv_D_train, Tweet_csv_D_test = train_test_split(Tweet_csv_D, test_size=0.2, random_state=42)

# Nettoyage des données (url, ponctuation, lemmatisation, ...)

In [ ]:
# On créer des dictionnaire contenant des dictionnaires sur les tweets

def concatenate_dictionaries(dict1, dict2):
  """Concatenates two dictionaries with incrementing indices.

  Args:
    dict1: The first dictionary.
    dict2: The second dictionary.

  Returns:
    A new dictionary containing the elements of both input dictionaries
    with incrementing indices.
  """

  max_key_dict1 = 0
  if dict1:  # Check if dict1 is not empty
    max_key_dict1 = max(int(key) for key in dict1.keys())

  result_dict = dict1.copy()  # Start with a copy of dict1

  for key, value in dict2.items():
    result_dict[str(int(key) + max_key_dict1 + 1)] = value
    # Increment key and add to result_dict

  return result_dict

def create_dict(Tweet_csv,dict_to_complete):
  """ create_dict créé un dictionnaire à partir d'un csv. Chaque clef est un index se référant à un tweet.
      La valeur du dictionnaire en cet index est un dictionnaire concernant le tweet
    :param create_dict: csv contenant les tweets, dictionnaire qui contiendra l'information sur le tweets
    :return: dictionnaire contenant les tweets
  """
  for index, row in Tweet_csv.iterrows():  # Iterate using iterrows()
    tweet_text = row['data__text']  # Directly access the text as a string
    dict_to_complete[index] = {'text': tweet_text}
  return

dict_combine, dict_L, dict_D = {}, {}, {}
dict_train_L, dict_train_D = {}, {}
dict_test_L, dict_test_D = {}, {}

create_dict(Tweet_csv_L,dict_L)
create_dict(Tweet_csv_D,dict_D)
create_dict(Tweet_csv_L_train,dict_train_L)
create_dict(Tweet_csv_D_train,dict_train_D)
create_dict(Tweet_csv_L_test,dict_test_L)
create_dict(Tweet_csv_D_test,dict_test_D)
dict_combine = concatenate_dictionaries(dict_L, dict_D)


In [ ]:
print(len(dict_combine), len(dict_L),len(dict_D))

17076 5636 11440


In [ ]:
url_pattern = re.compile(r'(https?://|www\.)\S+')
pattern=r'\w+|#[a-zA-Z0-9_àâäéèêëïîôöùûüçÀÂÄÉÈÊËÏÎÔÖÙÛÜÇ]+|@[a-zA-Z0-9_àâäéèêëïîôöùûüçÀÂÄÉÈÊËÏÎÔÖÙÛÜÇ]+|[^\w\s]+'
tokenizer = RegexpTokenizer(pattern)

def remove_urls(text):
    return url_pattern.sub('', text)

def remove_punctuation(text_without_urls):
  """ Fonction pour retirer la ponctuation des tweets, les stop_words et les tokeniser.
    :param remove_punctuation: String contenant le tweet
    :return: Liste des tokens
  """
  global dict_combine
  without_punctuation = []
  text_tokenise = tokenizer.tokenize(text_without_urls)
  for word in text_tokenise:
    if word.isalpha() or (word.startswith('#') and len(word_tokenize(word))>1) or (word.startswith('@') and len(word_tokenize(word))>1):
    # We import and use the stopwords from NLTK library
      if word.lower() not in stopwords.words('french'):
        without_punctuation.append(word.lower())
  return without_punctuation

def ajt_tokenize (dictio):
  for key in dictio.keys():
    text=dictio[key]['text']
    text_without_urls = remove_urls(text)
    dictio[key]['text_tokenise'] = remove_punctuation(text_without_urls)
  return

ajt_tokenize(dict_combine)
ajt_tokenize(dict_test_L)
ajt_tokenize(dict_test_D)
ajt_tokenize(dict_train_L)
ajt_tokenize(dict_train_D)

In [ ]:
# Pour s'assurer que 'text_tokenise" ne contient pas d'url, ponctuation,etc.
print(dict_combine[0])

{'text': "RT @LutteOuvriere: #Migrants, #chômage, #police : une société en crise, malade du capitalisme \\n L'édito ⬇️ \\n https://t.co/sxkwJdMZXX", 'text_tokenise': ['rt', '@lutteouvriere', '#migrants', '#chômage', '#police', 'société', 'crise', 'malade', 'capitalisme', 'édito']}


In [ ]:
def lemmatize_tokens_g(phrase_tokenisee):
    """
    Fonction pour lemmatiser les tokens d'une phrase tokenisée.

    :param phrase_tokenisee_g : Objet spaCy Doc contenant les tokens
    :return: Liste des mots lemmatisés
    """
    mots_lemmatises = []
    global mots_lem_g  # Accès à la liste globale pour stocker tous les mots lemmatisés

    j = 0
    while j < len(phrase_tokenisee):
        token = phrase_tokenisee[j]

        if token.text.startswith('#') and j + 1 < len(phrase_tokenisee):
            # Combinaison du hashtag avec le token suivant
            combined_text = token.text + phrase_tokenisee[j + 1].text
            combined_token = nlp(combined_text)
            mots_lemmatises.append(combined_token)
            mots_lem_g.append(combined_token)
            j += 2  # Passer au token après le suivant
        elif token.text.startswith('@'):
            # Lemmatisation des mentions
            mots_lemmatises.append(token.lemma_)
            mots_lem_g.append(token.lemma_)
            j += 1
        else :
            # Lemmatisation des autres mots
            mots_lemmatises.append(token.lemma_)
            mots_lem_g.append(token.lemma_)
            j += 1

    return mots_lemmatises

def lemmatize_tokens(phrase_tokenisee):
    """
    Fonction pour lemmatiser les tokens d'une phrase tokenisée.

    :param phrase_tokenisee: Objet spaCy Doc contenant les tokens
    :return: Liste des mots lemmatisés
    """
    mots_lemmatises = []
    global mots_lem_g  # Accès à la liste globale pour stocker tous les mots lemmatisés

    j = 0
    while j < len(phrase_tokenisee):
        token = phrase_tokenisee[j]

        if token.text.startswith('#') and j + 1 < len(phrase_tokenisee):
            # Combinaison du hashtag avec le token suivant
            combined_text = token.text + phrase_tokenisee[j + 1].text
            combined_token = nlp(combined_text)
            mots_lemmatises.append(combined_token)
            j += 2  # Passer au token après le suivant
        elif token.text.startswith('@'):
            # Lemmatisation des mentions
            mots_lemmatises.append(token.lemma_)
            j += 1
        else :
            # Lemmatisation des autres mots
            mots_lemmatises.append(token.lemma_)
            j += 1

    return mots_lemmatises

In [ ]:
# Liste globale pour stocker tous les mots lemmatisés, t exécution
mots_lem_g = []

def lemmatisation (dict_ou_ajouter):
  for index, value in dict_ou_ajouter.items():
    # Création de l'objet spaCy à partir de la liste 'text_tokenise'
    phrase_tokenisee = nlp(" ".join(value['text_tokenise']))
    # Appel de la fonction pour lemmatiser les tokens
    value['mots_lemmatises'] = lemmatize_tokens(phrase_tokenisee)
  return

def lemmatisation_g (dict_ou_ajouter):
  for index, value in dict_ou_ajouter.items():
    # Création de l'objet spaCy à partir de la liste 'text_tokenise
    phrase_tokenisee = nlp(" ".join(value['text_tokenise']))
    # Appel de la fonction pour lemmatiser les tokens
    value['mots_lemmatises'] = lemmatize_tokens_g(phrase_tokenisee)
  return

lemmatisation_g(dict_combine)
print("fait 1")
lemmatisation(dict_test_L)
print("fait 2")
lemmatisation(dict_test_D)
print("fait 3")
lemmatisation(dict_train_L)
print("fait 4")
lemmatisation(dict_train_D)

fait 1
fait 2
fait 3
fait 4


In [ ]:
# On vérifie que les mots sont biens lemmatisés
i=4
print(dict_combine[i]['text_tokenise'])
print(dict_combine[i]['mots_lemmatises'])

['principaux', 'candidats', 'gauche', 'droite', 'choisi', 'faire', 'terrorisme', 'migrants', 'centre', 'campagne', '#lt']
['principal', 'candidat', 'gauche', 'droite', 'choisir', 'faire', 'terrorisme', 'migrant', 'centre', 'campagne', #lt]


In [ ]:
#afficher les k n-grammes les plus communs dans le texte, ici les 100 premiers trigrammes
n=3
k=100
ngram = list(nltk.ngrams(mots_lem_g,n))
fdist = FreqDist(ngram)
print(fdist.most_common(k))

[(('rt', '@nicolasbay', '_'), 152), (('loi', 'asil', 'immigration'), 107), (('lutte', 'contre', 'immigration'), 87), (('lutter', 'contre', 'immigration'), 83), (('pompe', 'aspirant', 'immigration'), 73), (('contre', 'immigration', 'clandestin'), 71), (('traiter', 'libre', 'échange'), 67), (('mettre', 'fin', 'immigration'), 64), (('lien', 'entrer', 'immigration'), 58), (('emmanuel', 'macron', 'avoir'), 57), (('ce', 'immigration', 'massif'), 56), (('trafic', 'être', 'humain'), 54), (('politique', 'dissuasif', 'immigration'), 48), (('contre', 'submersion', 'migratoire'), 43), (('mettre', 'place', 'politique'), 42), (('contre', 'immigration', 'massif'), 41), (('mlp', 'rt', '@mlp_officiel'), 40), (('toujours', 'plus', 'immigration'), 39), (('rt', '@mlp_officiel', 'immigration'), 39), (('rt', '@mlp_officiel', 'avoir'), 37), (('@cnews', 'rt', '@mlp_officiel'), 36), (('projet', 'loi', 'asil'), 35), (('contre', 'immigration', 'illégal'), 35), (('rt', '@dlf_officiel', 'dupontaignan'), 35), (('fr

In [ ]:
# Calculate vocabulary size before lemmatization
vocab_before_lemmatization = set()
for key in dict_combine:
    vocab_before_lemmatization.update(dict_combine[key]['text_tokenise'])
print(f"Vocabulary size before lemmatization: {len(vocab_before_lemmatization)}")

# Calculate vocabulary size after lemmatization
vocab_after_lemmatization = set(mots_lem_g)
print(f"Vocabulary size after lemmatization: {len(vocab_after_lemmatization)}")

# ... (rest of your code)

Vocabulary size before lemmatization: 23713
Vocabulary size after lemmatization: 35668


# Création du vocabulaire finale

In [ ]:
les_n = [2,3]
for i in les_n:
  n=i
  ngram = list(nltk.ngrams(mots_lem_g,n))
  vocab_after_lemmatization.update(ngram)
print(f"Vocabulary size after lemmatization: {len(vocab_after_lemmatization)}")

Vocabulary size after lemmatization: 438867


## Ressources (normale si ça plante  ici)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

for key, value in dict_combine.items():
    tweet = value['text'].iloc[0]  # Access the tweet text
    tokens = word_tokenize(tweet)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    value['lemmas'] = lemmas # Add the lemmas to the dictionary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


AttributeError: 'str' object has no attribute 'iloc'

In [ ]:
#!python -m spacy download fr_core_news_md
import spacy
import nltk
from nltk import FreqDist
#spacy.load('fr_core_news_md') # No need to call spacy.load twice
# On va lématiser les mots
nlp = spacy.load('fr_core_news_md')
mots_lem_g=[]
for index in dict_combine.keys():
  mots_lemmatises=[]
  # Accessing the correct key 'text_tokenise'
  phrase_de_mots_tokenise=nlp(" ".join(dict_combine[index]['text_tokenise']))
  contenant = {}
  for i,w in enumerate(phrase_de_mots_tokenise):
    contenant[i]=w
    j=0
  while (j)!=len(phrase_de_mots_tokenise):
    w=contenant[j]
    if  w.text.startswith('#') and (j+1!=len(phrase_de_mots_tokenise)):
      w=w.text+phrase_de_mots_tokenise[j+1].text
      w=nlp(w)  # Get the first token from the processed text
      mots_lemmatises.append(w) # Append lemma of combined token
      mots_lem_g.append(w)
      j+=2
    elif w.text.startswith('@') and (j+1!=len(phrase_de_mots_tokenise)):
      mots_lemmatises.append(w.lemma_)
      mots_lem_g.append(w.lemma_)
      j+=1
    else:
      mots_lemmatises.append(w.lemma_)
      mots_lem_g.append(w.lemma_)
      j+=1

  dict_combine[index]['mots_lemmatises']=mots_lemmatises

In [ ]:
url_pattern = re.compile(r'(https?://|www\.)\S+')
pattern=r'\w+|#[a-zA-Z0-9_àâäéèêëïîôöùûüçÀÂÄÉÈÊËÏÎÔÖÙÛÜÇ]+|@[a-zA-Z0-9_àâäéèêëïîôöùûüçÀÂÄÉÈÊËÏÎÔÖÙÛÜÇ]+|[^\w\s]+'
tokenizer = RegexpTokenizer(pattern)
for key in dict_combine.keys():

  text=dict_combine[key]['text']
  text_without_urls = url_pattern.sub('url', text) # On enlève les urls du code

  # Personnalisation du tokenizer pour garder les # ou @ avec le mot

  text_tokenise = tokenizer.tokenize(text_without_urls)
  without_punctuation = []

  # On tokenize en enlevant les stop_words en gardant les mentions et tweets
  for word in text_tokenise:
    if word.isalpha() or (word.startswith('#') and len(word_tokenize(word))>1) or (word.startswith('@') and len(word_tokenize(word))>1):
      if word.lower() not in stopwords.words('french'):
        without_punctuation.append(word.lower())
  dict_combine[key]['text_tokenise'] = without_punctuation

In [ ]:
#!python -m spacy download fr_core_news_md
import spacy
import nltk
from nltk import FreqDist
#spacy.load('fr_core_news_md') # No need to call spacy.load twice
# On va lématiser les mots
nlp = spacy.load('fr_core_news_md')
mots_lem_g=[]
for index in dict_combine.keys():
  mots_lemmatises=[]
  # Accessing the correct key 'text_tokenise'
  phrase_de_mots_tokenise=nlp(" ".join(dict_combine[index]['text_tokenise']))
  for i,w in enumerate(phrase_de_mots_tokenise):
    if (w.text.startswith('@') or w.text.startswith('#')) and (i+1!=len(phrase_de_mots_tokenise)):
      w=w.text+phrase_de_mots_tokenise[i+1].text
      combined_token = nlp(w)[0]  # Get the first token from the processed text
      mots_lemmatises.append(combined_token.lemma_) # Append lemma of combined token
      mots_lem_g.append(combined_token.lemma_)
    else:
      mots_lemmatises.append(w.lemma_)
      mots_lem_g.append(w.lemma_)
  dict_combine[index]['mots_lemmatises']=mots_lemmatises